In [1]:
# Import dependencies
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

from config import db_password

In [2]:
# load the data
df = pd.read_csv("Resources/market_complete.csv")
df

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,purchases_sum,Lat,Long,Per Capita Income,Currency Conv to USD,Total_Dependents,Total_campaigns,age,edu_classes,relation_status
0,1826,1970,Graduation,Divorced,84835.0,0,0,2014-06-16,0,189,...,15,40.416775,-3.70379,"27,057.2",0.846231,0,0,44,2,4
1,5371,1989,Graduation,Single,21474.0,1,0,2014-04-08,0,6,...,8,40.416775,-3.70379,"27,057.2",0.846231,1,1,25,2,1
2,7348,1958,PhD,Single,71691.0,0,0,2014-03-17,0,336,...,17,40.416775,-3.70379,"27,057.2",0.846231,0,0,56,5,1
3,1991,1967,Graduation,Together,44931.0,0,1,2014-01-18,0,78,...,7,40.416775,-3.70379,"27,057.2",0.846231,1,0,47,2,2
4,5642,1979,Master,Together,62499.0,1,0,2013-12-09,0,140,...,12,40.416775,-3.70379,"27,057.2",0.846231,1,0,35,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2177,9940,1958,Graduation,Together,64961.0,0,1,2012-12-23,97,382,...,16,-25.731340,28.21837,"5,090.7",14.717459,1,0,56,2,2
2178,3406,1964,Graduation,Single,45989.0,0,1,2012-10-22,97,138,...,16,-25.731340,28.21837,"5,090.7",14.717459,1,0,50,2,1
2179,313,1968,Graduation,Widow,73455.0,0,0,2013-10-28,98,901,...,22,-25.731340,28.21837,"5,090.7",14.717459,0,1,46,2,5
2180,5871,1979,Master,Together,24401.0,0,0,2012-08-31,98,73,...,16,-25.731340,28.21837,"5,090.7",14.717459,0,0,35,4,2


In [25]:
# Create bins for age groups
bins = [0, 25, 35, 45, 55, 65, 74]
age_groups = ["<25","25-35","35-45","45-55","55-65", ">65"]
# Add a "age group" column to the dataframe
df["Age_Group"] = pd.cut(df["age"], bins, labels=age_groups)
df["Age_Group"]

0       35-45
1         <25
2       55-65
3       45-55
4       25-35
        ...  
2177    55-65
2178    45-55
2179    45-55
2180    25-35
2181    45-55
Name: Age_Group, Length: 2182, dtype: category
Categories (6, object): ['<25' < '25-35' < '35-45' < '45-55' < '55-65' < '>65']

In [5]:
# Create bins for income groups
bins = [0, 10000, 35000, 55000, 75000, 90000, 120000, 170000]
age_groups = ["<10k","10-35k","35-55k","55-75k","75-90k", "90-120k", ">120k"]
# Add a "age group" column to the dataframe
df["Income_Group"] = pd.cut(df["Income"], bins, labels=age_groups)
df["Income_Group"]

0       75-90k
1       10-35k
2       55-75k
3       35-55k
4       55-75k
         ...  
2177    55-75k
2178    35-55k
2179    55-75k
2180    10-35k
2181    35-55k
Name: Income_Group, Length: 2182, dtype: category
Categories (7, object): ['<10k' < '10-35k' < '35-55k' < '55-75k' < '75-90k' < '90-120k' < '>120k']

In [ ]:
# Export the df with 2 new columns into a CSV.
df.to_csv("Resources/marketing_groups_df.csv", index=False)

In [3]:
# Add the count of campaign 6 to Total_campaigns
df['Total_campaigns'] = df['Total_campaigns'] + df['Response']

In [4]:
# create a dataframe for country value counts
country_value_count = pd.DataFrame(df.Country.value_counts())

# create a dataframe a blank cmp_country_df with 2 columns
cmp_country_df = pd.DataFrame(columns=['country','total_customers'])

# assign the list of coutries in the country_value_count dataframe to the 'country' column of the cmp_country_df dataframe
cmp_country_df['country'] = country_value_count.index.to_list()

# assign the value counts in the country_value_count dataframe to the 'total_customers' column of the cmp_country_df dataframe
cmp_country_df['total_customers'] = country_value_count.Country.to_list()
cmp_country_df

,country,total_customers
0,SP,1054
1,SA,333
2,CA,263
3,AUS,158
4,IND,145
5,GER,120
6,US,109


In [5]:
# Retrieve the number of responders to each campaign for each country
cmps = ['AcceptedCmp1','AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5','Response']
sp = df[df.Country == 'SP'][cmps].sum().to_list()
sa = df[df.Country == 'SA'][cmps].sum().to_list()
ca = df[df.Country == 'CA'][cmps].sum().to_list()
aus = df[df.Country == 'AUS'][cmps].sum().to_list()
ind = df[df.Country == 'IND'][cmps].sum().to_list()
ger = df[df.Country == 'GER'][cmps].sum().to_list()
us = df[df.Country == 'US'][cmps].sum().to_list()

In [6]:
# append the above values to the cmp_country_df dataframe
cmp_country_df[['cmp1_response','cmp2_response', 'cmp3_response', 'cmp4_response', 'cmp5_response','cmp6_response']] = [sp,sa,ca,aus,ind,ger,us]
cmp_country_df

,country,total_customers,cmp1_response,cmp2_response,cmp3_response,cmp4_response,cmp5_response,cmp6_response
0,SP,1054,76,15,80,87,86,170
1,SA,333,20,4,21,20,21,52
2,CA,263,18,6,17,24,21,38
3,AUS,158,6,0,9,6,12,22
4,IND,145,7,2,13,11,6,12
5,GER,120,7,2,10,11,8,17
6,US,109,7,0,8,6,5,13


In [7]:
# Retrieve the number of customers who responded to at least 1 campaign for each country
c1=df[df.Country == 'SP'][df.Total_campaigns > 0]['ID'].count()
c2=df[df.Country == 'SA'][df.Total_campaigns > 0]['ID'].count()
c3=df[df.Country == 'CA'][df.Total_campaigns > 0]['ID'].count()
c4=df[df.Country == 'AUS'][df.Total_campaigns > 0]['ID'].count()
c5=df[df.Country == 'IND'][df.Total_campaigns > 0]['ID'].count()
c6=df[df.Country == 'GER'][df.Total_campaigns > 0]['ID'].count()
c7=df[df.Country == 'US'][df.Total_campaigns > 0]['ID'].count()

C:\Users\HP\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\HP\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\HP\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\Users\HP\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\HP\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\HP\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:7: 

In [8]:
# add a column called 'cmps_response' whose values are the number of customers who responded to at least 1 campaign
cmp_country_df['cmps_response'] = [c1,c2,c3,c4,c5,c6,c7]
cmp_country_df

,country,total_customers,cmp1_response,cmp2_response,cmp3_response,cmp4_response,cmp5_response,cmp6_response,cmps_response
0,SP,1054,76,15,80,87,86,170,306
1,SA,333,20,4,21,20,21,52,89
2,CA,263,18,6,17,24,21,38,70
3,AUS,158,6,0,9,6,12,22,31
4,IND,145,7,2,13,11,6,12,34
5,GER,120,7,2,10,11,8,17,34
6,US,109,7,0,8,6,5,13,28


In [9]:
# Export the cmp_country_df dataframe into a CSV file.
cmp_country_df.to_csv("Resources/cmp_country.csv", index=False)